In [ ]:
import random

import numpy as np
import pandas as pd

import os
from os import listdir
from os.path import join, splitext

import cv2
from PIL import Image

import torch
from torch import nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, ToTensor, ToPILImage

import matplotlib.pyplot as plt

SRGAN Model Libraries

In [ ]:
from srgan.model import Generator, Discriminator
from srgan.loss import GeneratorLoss

Iterative Crowd Counting Model Libraries

In [ ]:
from icc.data_loaderB import ImageDataLoader
from icc.model_ic_CNN import modelicCNN, retrain_icCNN
from icc.evaluate_icCNN import evaluate_model
from icc import network

GPU Configurations

In [ ]:
# GPU to run on
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [ ]:
# Fixing random seed
rand_seed = 26700

np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
torch.cuda.manual_seed(rand_seed)

Hyperparameters to tune

In [ ]:
BATCH_SIZE = 2
ALPHA = 2
MAX_EPOCH = 200

LR = 0.00001
WEIGHT_DECAY = 0.00001
MOMENTUM = 0.9

W = 0.01

DataLoader Configuration

In [ ]:
train_path = 'data/train/images'
train_gt_path = 'data/train/ground_truth_csv'

val_path = 'data/val/images'
val_gt_path = 'data/val/ground_truth_csv'

output_dir = 'logs/model_icCNN/'

In [ ]:
train_data_loader = ImageDataLoader(train_path, 
                                    train_gt_path,
                                    shuffle=True,
                                    gt_downsample=False,
                                    pre_load=True,
                                    sr_mode=True)

val_data_loader = ImageDataLoader(val_path, 
                                  val_gt_path,
                                  shuffle=False,
                                  gt_downsample=False,
                                  pre_load=False,
                                  sr_mode=True)

Utils to read and transform data

In [ ]:
def RandomCrop(Input, Density, h, w, th, tw):
    x1 = random.randint(0, h - th)
    y1 = random.randint(0, w - tw)

    Input = Input[x1:x1 + th, y1:y1 + tw]
    Density = Density.reshape((h, w))[x1:x1 + th, y1:y1 + tw]

    return Input, Density

In [ ]:
def LowerResolution(img):
    y, x = img.shape[0], img.shape[1]
    fx, fy = int(x // ALPHA), int(y // ALPHA)
    
    img_small = cv2.resize(img, (fx, fy), interpolation=cv2.INTER_CUBIC)
    lr_img = cv2.resize(img_small, (x, y), interpolation=cv2.INTER_CUBIC)
        
    return lr_img

In [ ]:
def get_training_batch(blob):
    img = blob['data']
    gt_density = blob['gt_density']

    h = img.shape[0]
    w = img.shape[1]
    
    th = int(h/3.0 - ((h/3.0) % 4))
    tw = int(w/3.0 - ((w/3.0) % 4))

    Input_HR = torch.zeros(BATCH_SIZE, 3, th, tw)
    Input_LR = torch.zeros(BATCH_SIZE, 3, th, tw)
    GT_Density = torch.zeros(BATCH_SIZE, 1, th, tw)

    for cur_step in range(0, BATCH_SIZE):
        img_crop, gt_density_crop = RandomCrop(img, gt_density, h, w, th, tw)

        lr_img = LowerResolution(img_crop)

        Input_HR[cur_step] = ToTensor()(img_crop)
        Input_LR[cur_step] = ToTensor()(lr_img)
        GT_Density[cur_step] = torch.from_numpy(gt_density_crop)
        
    return Input_HR, Input_LR, GT_Density

Initializing SRGAN parameters

In [ ]:
# Initializing model
netG = Generator()
netD = Discriminator()

network.weights_normal_init(netG)
network.weights_normal_init(netD)

netG.cuda()
netD.cuda()

netG.train()
netD.train()

# Initializing loss
gen_criterion = GeneratorLoss()
gen_criterion.cuda()

# Initializing optimizer
optimizerG = optim.Adam(netG.parameters())
optimizerD = optim.Adam(netD.parameters())

Initializing ICC parameters

In [ ]:
# Initializing model
net = modelicCNN()
network.load_net('model_twoBranchSimple/best_icCNN.h5', net)
net.cuda()
net.train()

# Initializing optimizer
optimizerN = torch.optim.Adam(net.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

# Initializing loss
net_loss_fn = nn.MSELoss()

Training Networks

In [ ]:
best_maeHR = float('inf') #sys.maxint
best_epochHR = 1

In [ ]:
G_Loss = []
D_Loss = []
N_Loss = []

In [ ]:
for epoch in range(1, MAX_EPOCH+1):

    count = 0
    g_epoch_loss = 0
    d_epoch_loss = 0
    n_epoch_loss = 0
    
    for blob in train_data_loader:
        
        optimizerN.zero_grad()
        optimizerD.zero_grad()
        optimizerG.zero_grad()

        Input_HR, Input_LR, GT_Density = get_training_batch(blob)
        
        Input_HR = Input_HR.cuda()
        Input_LR = Input_LR.cuda()
        GT_Density = GT_Density.cuda()
        
        Input_SR = netG(Input_LR)
        
        # https://stackoverflow.com/questions/12201577/how-can-i-convert-an-rgb-image-into-grayscale-in-python
        Input_SR_Gray = 255 * (0.299 * Input_SR[:,0,:,:] + 0.587 * Input_SR[:,1,:,:] + 0.114 * Input_SR[:,2,:,:])
        Input_SR_Gray = Input_SR_Gray[:,None,:,:]
        Input_SR_Gray = Input_SR_Gray.cuda()
                    
        Density = net(Input_SR_Gray)
        
        # Optimizing loss for icc net
        LossHR = net_loss_fn(Density, GT_Density)
        n_loss = 1000.0 * LossHR
        n_loss.backward(retain_graph=True)
        optimizerN.step()
        
        # Optimizing loss for Discriminator
        real_out = netD(Input_HR).mean()
        fake_out = netD(Input_SR).mean()
        d_loss = 1 - real_out + fake_out
        d_loss.backward(retain_graph=True)
        optimizerD.step()

        # Optimizing loss for Generator
        g_loss = W * gen_criterion(fake_out, Input_SR, Input_HR) + n_loss
        g_loss.backward()
        optimizerG.step()
        
        count += 1
        
        print("Training epoch {}, Batch_Num {}/{}, G_Loss {}, D_Loss {}, N_Loss {}".format(
            epoch, count, train_data_loader.get_num_samples(), g_loss, d_loss, n_loss))
        
        g_epoch_loss += g_loss.data.item()
        d_epoch_loss += d_loss.data.item()
        n_epoch_loss += n_loss.data.item()
        
        del Input_HR, Input_LR, GT_Density, Density, real_out, fake_out, d_loss, g_loss
        
    G_Loss.append(g_epoch_loss/count)
    D_Loss.append(d_epoch_loss/count)
    N_Loss.append(n_epoch_loss/count)
        
    if (epoch % 5 == 0):
        maeHR, mseHR = evaluate_model(net, netG, val_data_loader)
        
        net.train()
        netG.train()
        
        if maeHR < best_maeHR:
            best_maeHR = maeHR
            best_mseHR = mseHR
            
            best_epochHR = epoch
            
            torch.save(netG, os.path.join(output_dir, 'best_generator.pth'))
            network.save_net(os.path.join(output_dir, 'best_icCNN.h5'), net)

        print("EPOCH: %d, MAE_HR: %.1f, MSE_HR: %0.1f" % (epoch, maeHR, mseHR))
        print("BEST MAE_HR: %0.1f, BEST MSE_HR: %0.1f, BEST Epoch: %4.2f" % (best_maeHR, best_mseHR, best_epochHR))        

In [ ]:
torch.save(netG, os.path.join(output_dir, 'last_epoch_generator.pth'))
network.save_net(os.path.join(output_dir, 'last_epoch_icCNN.h5'), net)

In [ ]:
print(G_Loss)

In [ ]:
print(D_Loss)

In [ ]:
print(N_Loss)

In [ ]:
# Code to output the Super Resolved images.

# val_data_loader = ImageDataLoader(val_path, 
#                                   val_gt_path,
#                                   shuffle=False,
#                                   gt_downsample=False,
#                                   pre_load=False,
#                                   sr_mode=True)

# model = torch.load(os.path.join(output_dir, 'netG.pth'))
# model.cuda()

# out_path = "logs/SRGAN/val/"

# for blob in val_data_loader:
#     img = blob['data']
#     fname = blob['fname']
    
#     img = Variable(ToTensor()(img), requires_grad=False).unsqueeze(0)
#     img = img.cuda()
    
#     out = model(img)
#     out = ToPILImage()(out[0].data.cpu())
    
#     out.save(join(out_path, 'out_' + fname))